In [1]:
import os
import sys
from PyQt5 import QtGui
from PyQt5.QtCore import *
from PyQt5.QtGui import *
from PyQt5.QtWidgets import *
from PyQt5.QtMultimedia import *
from PyQt5.QtMultimediaWidgets import *

import numpy as np
import pandas as pd
from scipy.optimize import curve_fit, minimize
from scipy.stats import gaussian_kde

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg, NavigationToolbar2QT as NavigationToolbar
from matplotlib.figure import Figure
from matplotlib import gridspec
from mpl_toolkits.mplot3d import Axes3D
matplotlib.use('Qt5Agg')

In [23]:
class MplCanvas(FigureCanvasQTAgg):

    def __init__(self, parent=None, width=5, height=4, dpi=100, is3d=False):
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = fig.add_subplot(111, projection='3d' if is3d else None)
        super(MplCanvas, self).__init__(fig)

class MainWindow(QWidget):

    def __init__(self):
        super(MainWindow, self).__init__()
        self.initUI()
        
    def initUI(self):

        tablebox = QVBoxLayout()
        table = QTableWidget()
        table.setColumnCount(2)
        table.setRowCount(17)
        table.setHorizontalHeaderLabels(['Radius', 'Thickness Factor'])
        for i in np.arange(0, 170, 10):
            table.setItem(i//10, 0, QTableWidgetItem(str(i)))
        table.cellChanged.connect(self.onCellChanged)
        tablebox.addWidget(table)

        tablewidget = QWidget()
        tablewidget.setLayout(tablebox)

        pltbox1 = QVBoxLayout()
        ax1 = MplCanvas(self, width=4, height=4, dpi=100)
        x = np.linspace(0, 10, 1000)
        y = np.sin(x)
        ax1.axes.plot(x, y)
        toolbar1 = NavigationToolbar(ax1, self)
        pltbox1.addWidget(toolbar1)
        pltbox1.addWidget(ax1)

        pltbox2 = QVBoxLayout()
        ax2 = MplCanvas(self, width=4, height=4, dpi=100, is3d=True)
        x = np.arange(-5, 5, 0.1)
        y = np.arange(-5, 5, 0.1)
        X, Y = np.meshgrid(x, y)
        Z = np.sin(np.sqrt(X**2 + Y**2))
        ax2.axes.plot_surface(X, Y, Z, cmap='viridis')
        toolbar2 = NavigationToolbar(ax2, self)
        pltbox2.addWidget(toolbar2)
        pltbox2.addWidget(ax2)

        pltwidget = QWidget()
        pltwidbox = QHBoxLayout()
        pltwid1 = QWidget()
        pltwid1.setLayout(pltbox1)
        pltwid1.setFixedWidth(500)
        pltwid2 = QWidget()
        pltwid2.setLayout(pltbox2)
        pltwid2.setFixedWidth(500)
        pltwidbox.addWidget(pltwid1)
        pltwidbox.addWidget(pltwid2)
        pltwidget.setLayout(pltwidbox)
        pltwidget.setFixedWidth(1000)

        box = QHBoxLayout()
        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(tablewidget)
        splitter.addWidget(pltwidget)
        box.addWidget(splitter)
        self.setLayout(box)

        QApplication.setStyle(QStyleFactory.create('Plastique'))
        self.setWindowTitle("Thickness Profile Plotter")
        self.show()

    def onCellChanged(self, row, col):
        print(row, col)



    def closeEvent(self, event):
        QApplication.quit()


app = QCoreApplication.instance()
if app is None:
    app = QApplication([])
window = MainWindow()
window.show()

try:
    from IPython.lib.guisupport import start_event_loop_qt5
    start_event_loop_qt5(app)
except ImportError:
    app.exec_()


In [3]:
class MplCanvas(FigureCanvasQTAgg):

    def __init__(self, parent=None, width=5, height=4, dpi=100):
        fig = Figure(figsize=(width, height), dpi=dpi)
        self.axes = fig.add_subplot(111)
        super(MplCanvas, self).__init__(fig)

class MainWindow(QMainWindow):

    def __init__(self, *args, **kwargs):
        super(MainWindow, self).__init__(*args, **kwargs)

        sc = MplCanvas(self, width=5, height=4, dpi=100)
        sc.axes.plot([0,1,2,3,4], [10,10,20,3,40])

        # Create toolbar, passing canvas as first parament, parent (self, the MainWindow) as second.
        toolbar = NavigationToolbar(sc, self)

        layout = QVBoxLayout()
        layout.addWidget(toolbar)
        layout.addWidget(sc)

        # Create a placeholder widget to hold our toolbar and canvas.
        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)

        self.show()

    def closeEvent(self, event):
        QApplication.quit()


app = QCoreApplication.instance()
if app is None:
    app = QApplication([])
window = MainWindow()
window.show()

try:
    from IPython.lib.guisupport import start_event_loop_qt5
    start_event_loop_qt5(app)
except ImportError:
    app.exec_()


In [4]:
class MyWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        layout = QVBoxLayout()
        self.table = QTableWidget()
        self.button = QPushButton("Paste Table")
        layout.addWidget(self.table)
        layout.addWidget(self.button)

        central_widget = QWidget()
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.button.clicked.connect(self.paste_table)

    def paste_table(self):
        clipboard = QApplication.clipboard()
        table_data = clipboard.mimeData().text().split('\n')  # Get the table data from clipboard

        row_count = len(table_data)
        col_count = len(table_data[0].split('\t'))

        self.table.setRowCount(row_count)
        self.table.setColumnCount(col_count)

        for i, row in enumerate(table_data):
            columns = row.split('\t')
            for j, column in enumerate(columns):
                item = QTableWidgetItem(column)
                self.table.setItem(i, j, item)

    def closeEvent(self, event):
        QApplication.quit()
        

app = QCoreApplication.instance()
if app is None:
    app = QApplication([])
window = MyWindow()
window.show()

try:
    from IPython.lib.guisupport import start_event_loop_qt5
    start_event_loop_qt5(app)
except ImportError:
    app.exec_()